## Compute tick count using dedicated service - Python

### Overview
Tick count can be used as an indicator to illustrate market activity by computing the technical transactions/transmissions count on a tick by tick basis.

It represents a valuable technical indicator to monitor tick by tick data reception. Building a history of tick count can be useful in the following use-cases:
* guarantee tick by tick data good reception and completion
* compare trading periods and highlight high-level of activity periods
* combined with other indicators, it helps to optimal future rolling 
* etc.

Correlated with a set of other indicators such as traded volume and count of trades, over the same look back period, it repserents a good measurement of the market quality.

This samples enables to retrieve **on-demand** tick count data points by calling a dedicated service.

### Inputs/outputs
Tick count sample requires instrument's identifier, date time intervals and time granularity as per inputs. It returns the tick count sampled using the input time granularity.

This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

<div class="alert alert-block alert-info">
    <b>Note:</b> A similar scenario dedicated to future contract's tick count, all maturities covered, is available in <a href="https://ganymede.systemathics.com/user/talel.nasri@systemathics.com/lab/tree/csharp/future_roll.ipynb">future roll strategy</a> notebook.
</div>

### Services used
This sample uses *gRPC requests* in order to retrieve ticks from the dedicated hosted service. The queried endpoint in this script are:
* TopologiesService: to directly retrieve ticks objects from the server.

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.topology.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package

***

# Run market activity sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib

In [ ]:
pip install systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import date
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.topology.v1.topologies_pb2 as topologies
import systemathics.apis.services.topology.v1.topologies_pb2_grpc as topologies_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *TopologiesService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Topology request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Topology parmeters

In [ ]:
# set topology time granularity (daily, weekly...)
granularity = topologies.TOPOLOGY_GRANULARITY_DAILY

# set level: Trades or Trades and Book
my_level = level.LEVEL_TRADES_AND_BOOK

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and ensure that the reply is not empty:

In [ ]:
# generate the topologies request
request = topologies.TopologiesRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    granularity = granularity,
    level = my_level
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the topologies service
        service = topologies_service.TopologiesServiceStub(channel)
        
        # process the topologies request
        response = service.Topologies(
            request=request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# get tick count data
print("Total entries: ", len(response.entries))

ticks = 0
for entry in response.entries:
    ticks = ticks + entry.ticks_count
    
print("Total ticks count retreieved: ", ticks)

### Step 4: Visualize data

#### 4.1 Retrieve tick count data

In [ ]:
# create the market activitby pandasdataframe
counts = [entry.ticks_count for entry in response.entries]
dates = [datetime(year=entry.begin.year,day=entry.begin.day, month=entry.begin.month) for entry in response.entries]

d = {'Date': dates, 'Count': counts}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot tick count
The following code snippets diplay tick count's request results with the package of your choice.<br>*`matplotlib`* is used as per open source display package. Plot a simple line graph with the closing price and volume to easily spot spikes in trading as follows:

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.bar(dates,counts)
plt.xlabel("Date",size="20")
plt.ylabel("Tick count",size="20")
plt.title("Tick count for {0}-{1} Total ticks count: {2}".format(ticker,exchange, ticks),size="20")
plt.show()